In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:

def sql_query(query):
    return pd.io.sql.read_sql(query, con = engine)

### Вывод таблиц

In [3]:

books_query=''' 
select *
from books
where book_id=299
limit 5
'''

authors_query='''
select *
from authors
limit 5
'''

publishers_query=''' 
select *
from publishers
limit 5
'''

ratings_query=''' 
select *
from ratings
limit 5
'''

reviews_query=''' 
select *
from reviews
--where book_id=299
--limit 5
'''

In [4]:
display(sql_query(books_query))
display(sql_query(authors_query))
display(sql_query(publishers_query))
display(sql_query(ratings_query))
display(sql_query(reviews_query))


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,299,236,Harry Potter and the Chamber of Secrets (Harry...,341,1999-06-02,24


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


### Количество книг вышедших после 1 января 2000 года;

In [5]:
count_books='''
select count(book_id) as cnt_books
from books
where publication_date>'2000-01-01'

'''
print('Количество книг вышедших после 1 января 2000 года: ',sql_query(count_books).loc[0]['cnt_books'])

Количество книг вышедших после 1 января 2000 года:  819


### Количество обзоров и средняя оценка книг;

In [6]:
mean_rating='''
select

    b.title,
    b.book_id,
    case
    when rv.cnt_review is null then 0 else cnt_review end as cnt_review,
    case
    when r.avg_rating is null then 0 else r.avg_rating end as avg_rating
    
    

from
    books b 
    left join (select
                    book_id,
                    count(review_id) as cnt_review
                from 
                    reviews
                group by
                    book_id) as rv
                            on b.book_id=rv.book_id
    left join (select
                    book_id,
                    avg(rating) as avg_rating
                from 
                    ratings
                group by
                    book_id) as r
                             on rv.book_id=r.book_id
--group by
   -- b.title
order by
   cnt_review desc,
   avg_rating desc

limit 10    
'''


In [7]:
sql_query(mean_rating)


,title,book_id,cnt_review,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.287500
3,The Book Thief,656,6,4.264151
4,The Glass Castle,734,6,4.206897
5,Outlander (Outlander #1),497,6,4.125000
6,The Hobbit or There and Back Again,750,6,4.125000
7,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
8,The Lightning Thief (Percy Jackson and the Oly...,779,6,4.080645
9,Water for Elephants,963,6,3.977273


### Издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [9]:
publisher_more_50pages='''
select
    publishers.publisher,
    count(distinct books.book_id) as cnt_books
from
    books
    left join publishers
        on books.publisher_id=publishers.publisher_id
where
    books.num_pages>50
group by
     publishers.publisher
order by
    cnt_books desc
Limit
    1
'''


In [10]:
print('Издательсво,выпустившее наибольшее число книг толще 50 страниц: ',sql_query(publisher_more_50pages).loc[0]['publisher'])

Издательсво,выпустившее наибольшее число книг толще 50 страниц:  Penguin Books


`Penguin Books`

Британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном. Главной заслугой издательства считают демократизацию книжного рынка, превратившую книгу из предмета роскоши в удовольствие, доступное массам

### Автор с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [15]:
i='''

with book_mark as( 
select
    book_id
--  count(rating_id) as cnt_books
from
    ratings
group by
    book_id
having
    count(rating_id)>50)

select
    authors.author,
    AVG(rating) as avg_rating


from
    ratings
        left join books
            on ratings.book_id=books.book_id
        left join authors
            on books.author_id=authors.author_id
            
where
    ratings.book_id in (select book_id from book_mark)
group by
    authors.author
            
            
order by
   avg_rating desc

'''

In [19]:

print('Автор с самой высокой средней оценкой книг: ', sql_query(i).loc[0]['author'])





Автор с самой высокой средней оценкой книг:  J.K. Rowling/Mary GrandPré


`J.K. Rowling`

Британская писательница, сценаристка и кинопродюсер, наиболее известная как автор серии романов о Гарри Поттере. Книги о Гарри Поттере получили несколько наград и были проданы в количестве более 500 миллионов экземпляров. Они стали самой продаваемой серией книг в истории и основой для серии фильмов, ставшей третьей по кассовому сбору серией фильмов в истории.

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок


In [13]:
mean_review='''
with book_user as( 
select
    username
from
    ratings
group by
    username
having
    count(rating_id)>50)
    

select
    to_char(avg(cnt_review)::float,'99.00') as avg_count_rew
from (
        select
            reviews.username,
            count(reviews.username) as cnt_review
        from
            book_user
                left join reviews
                    on book_user.username=reviews.username
        group by
            reviews.username) as cnt_review
        
        '''

In [14]:
print('Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: ', sql_query(mean_review).loc[0]['avg_count_rew'])

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок:   24.33


### Вывод 
C 1 января 2000 года вышло 819 книг, самым популярным жанром является жанр фэнтази, книги этого жанра получают, большего всего обзоров и высокую среднюю оценку. Самым популярным писателем является  J.K. Rowling/Mary GrandPré. Издательством которое больше всего выпустило книг является Penguin Books. Что примечательно, что самый популярный писатель и издательтсво относятся к Великобретании, что свидетельсвует о высокой литературной культуре этой страны. Пользователи которые поставили более 50 оценок, в среднем делают 24,33 обзора. 